# Basic usefull panda Dataframe functions

In [ ]:
# ExtractLayer takes a chipID and returns its half and layer
def ExtractHalfLayer(ChipID):
    return Chip_DF.at[ChipID+1, 'Half'], Chip_DF.at[ChipID,'Layer']

# ExtractLayer takes a chipID and returns its layer
def ExtractLayer(ChipID):
    return int(Chip_DF.at[ChipID+1,'Layer'][5])

# BuildLayerDF returns a sub-dataframe of Chip_DF with the chips on "layer"
def BuildLayerDF(layer, DF = Chip_DF):
    DF_name = layer
    Sub_DF = DF[(Chip_DF['Layer'] == layer)]
    return Sub_DF

# BuildHalfLayerDF returns a sub-dataframe of Chip_DF with the chips on "layer" and "half"
def BuildHalfLayerDF(half,layer):
    DF_name = half + '_' + layer
    Sub_DF = Chip_DF[(Chip_DF['Half'] == half) & (Chip_DF['Layer'] == layer)]
    return [DF_name, Sub_DF]

# This functions builds a dataframe for a single 'chip1' on layer9 with all the chips in the cone generated by 'chip1' with their overlap
def BuildOverlapDF(chip=0,chipID=[], layer='0', X=0, Y=0, Z=0, Overlap=0):
    d = {'Layer9 Chip':chip,'ChipID': chipID, 'Layer':layer, 'X':X, 'Y':Y, 'Z':Z, 'Overlap':Overlap}
    return pd.DataFrame(data=d)
    
def BuildPEffDF(ListChipID, ListHalf, L0Eff=1, L1Eff=1, L2Eff=1, L3Eff=1, L4Eff=1, 
                L5Eff=1, L6Eff=1, L7Eff=1, L8Eff=1):
    Peff = L0Eff*L1Eff*L2Eff*L3Eff*L4Eff*L5Eff*L6Eff*L7Eff*L8Eff
    Data = {'ChipID':ListChipID, 'Half':ListHalf, 'Layer0':L0Eff, 'Layer1':L1Eff, 'Layer2':L2Eff, 'Layer3':L3Eff, 
            'Layer4':L4Eff, 'Layer5':L5Eff, 'Layer6':L6Eff, 'Layer7':L7Eff, 'Layer8':L8Eff, 'Pseaudo_Eff':Peff}
    return pd.DataFrame(data=Data)

def CreateDirectory(directory_name):
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)
        
def get_pseaudo_eff(chip_id, file_path):
    """
    Get the value under 'Pseaudo_Eff' corresponding to the given ChipID from the text file.
    """
    with open(file_path, 'r') as file:
        next(file)  # Skip the header line
        for line in file:
            data = line.strip().split()
            
            if data[0] == str(chip_id):
                return float(data[-1])  # Return the value under 'Pseaudo_Eff'
    # If ChipID is not found
    return 0

def get_weight(config, file_path):
    with open(file_path, 'r') as file:
        next(file)
        for line in file:
            data = line.strip().split()
            if data[0] == config:
                return float(data[-1])
# Testing if there are at least 2 clusters on D3 and D4
def Sub_Condition(layers):
    if 6 in layers and 7 in layers:
        return True
    elif 6 in layers and 8 in layers:
        return True
    elif 6 in layers and 9 in layers:
        return True
    elif 7 in layers and 8 in layers:
        return True
    elif 7 in layers and 9 in layers:
        return True
    elif 8 in layers and 9 in layers:
        return True
    else:
        return False
    
def LayerCombinations_C(n_cluster):
    Layers = ['layer0', 'layer1', 'layer2', 'layer3', 'layer4', 'layer5','layer6', 'layer7', 'layer8','layer9']
    """
    Generate all possible lists of length n_cluster from the elements of Layers
    where at least 4 disks are represented and there are at least 2 clusters on D3 and/or D4
    """
    # Generate all combinations
    all_combinations = [list(comb) for comb in combinations(Layers, n_cluster)]
    
    # Filter combinations where at least 4 disks are represented
    valid_combinations = []
    for combination in all_combinations:
        disks = set()
        config = []
        for layer in combination:
            l = int(layer[-1])
            config.append(l)
            disk_number = l // 2
            disks.add(disk_number)
        if len(disks) >= 4 and len(combination)>=5 and Sub_Condition(config):
            valid_combinations.append(combination)

    return valid_combinations